In [5]:
from regelum.system import System
from regelum.simulator import SciPy
from regelum.simulator import CasADi
from regelum.scenario import Scenario
from regelum.policy import Policy
from regelum import callback

import matplotlib.pyplot as plt
import numpy as np

class KinematicPoint(System):
    _name = "kinematic-point"
    _system_type = "diff_eqn"
    _dim_state = 2
    _dim_inputs = 2
    _dim_observation = 2
    _observation_naming = _state_naming = ["x", "y"]
    _inputs_naming = ["v_x", "v_y"]
    _action_bounds = [[-10.0, 10.0], [-10.0, 10.0]]

    # receive_action

    def _compute_state_dynamics(self, time, state):
        return self.inputs  # The velocity inputs directly define the rate of change of position.
    
class PDController(Policy):
    def __init__(
        self,
        system: KinematicPoint,
        sampling_time: float,
    ):
        super().__init__()
        self.system = system
        self.sampling_time = sampling_time
        
        self.pd_coefs: list[float] = [
            1,
            0.1,
        ]
        
    def get_action(self, observation):
        print("Observation: ", observation)

        
        print("Action:", np.array([[-observation[0, 0], -observation[0, 1]]]))
        # action = 0
        
        return np.array([-observation[0, 0], -observation[0, 1]])

# Define the initial state (initial position of the kinematic point).
initial_state = np.array([2.0, 2.0])  # Start at position (2, 2)

# Initialize the kinematic point system.
kinematic_point = KinematicPoint()

# Instantiate a simulator for the kinematic point system.
simulator =  SciPy(
    system=kinematic_point, state_init=initial_state, time_final=4, max_step=0.1
)

scenario = Scenario(
    policy=PDController(
        system=kinematic_point,
        sampling_time=0.01
    ),
    simulator=simulator,
    sampling_time=0.01,
    N_episodes=5,
    N_iterations=5,
)

scenario.run()

Observation:  [[2. 2.]]
Action: [[-2. -2.]]
Observation:  [[1.97777818 1.97777818]]
Action: [[-1.97777818 -1.97777818]]
Observation:  [[1.77979782 1.77979782]]
Action: [[-1.77979782 -1.77979782]]
Observation:  [[1.60001353 1.60001353]]
Action: [[-1.60001353 -1.60001353]]
Observation:  [[1.43837352 1.43837352]]
Action: [[-1.43837352 -1.43837352]]
Observation:  [[1.29306289 1.29306289]]
Action: [[-1.29306289 -1.29306289]]
Observation:  [[1.16243215 1.16243215]]
Action: [[-1.16243215 -1.16243215]]
Observation:  [[1.04499829 1.04499829]]
Action: [[-1.04499829 -1.04499829]]
Observation:  [[0.9394281 0.9394281]]
Action: [[-0.9394281 -0.9394281]]
Observation:  [[0.84452306 0.84452306]]
Action: [[-0.84452306 -0.84452306]]
Observation:  [[0.75920574 0.75920574]]
Action: [[-0.75920574 -0.75920574]]
Observation:  [[0.68250753 0.68250753]]
Action: [[-0.68250753 -0.68250753]]
Observation:  [[0.61355771 0.61355771]]
Action: [[-0.61355771 -0.61355771]]
Observation:  [[0.55157349 0.55157349]]
Action: 

In [3]:
from regelum.system import System
from regelum.simulator import SciPy
from regelum.simulator import CasADi

import matplotlib.pyplot as plt
import numpy as np


class KinematicPoint(System):
    _name = "kinematic-point"
    _system_type = "diff_eqn"
    _dim_state = 2
    _dim_inputs = 2
    _dim_observation = 2
    _observation_naming = _state_naming = ["x", "y"]
    _inputs_naming = ["v_x", "v_y"]
    _action_bounds = [[-10.0, 10.0], [-10.0, 10.0]]

    def _compute_state_dynamics(self, time, inputs):
        return inputs  # The velocity inputs directly define the rate of change of position.

def get_action(state):
    return (
        -state
    )  # Stabilizing action: control input is the negative of the system state.

# Define the initial state (initial position of the kinematic point).
initial_state = np.array([2.0, 2.0])  # Start at position (2, 2)

# Initialize the kinematic point system.
kinematic_point = KinematicPoint()

# Instantiate a simulator for the kinematic point system.
simulator =  SciPy(
    system=kinematic_point, state_init=initial_state, time_final=4, max_step=0.1
)

state_history = [initial_state.flatten()]  # Store the initial state.
times = [0.0]

for _ in range(int(simulator.time_final / simulator.max_step)):
    action = get_action(
        simulator.state
    )  # Compute the action based on the current state.
    simulator.receive_action(action)  # Provide the action to the simulator.
    simulator.do_sim_step()  # Perform one simulation step.
    state_history.append(simulator.state.flatten())  # Store the state after the step.
    times.append(simulator.time)

state_history = np.array(state_history)  # Convert history to numpy array for plotting.
times = np.array(times)  # Convert history to numpy array for plotting.

print(state_history)

fig, (ax_x, ax_y, ax_theta) = plt.subplots(1, 3, figsize=(12, 4))

ax_x.plot(times, state_history[:, 0], marker="o", markersize=4)
ax_x.set_title("X Position [m]")
ax_x.set_xlabel("Time [s]")
ax_x.grid(True)

ax_y.plot(times, state_history[:, 1], marker="o", markersize=4)
ax_y.set_title("Y Position [m]")
ax_y.set_xlabel("Time [s]")
ax_y.grid(True)

fig.savefig("run_results.png")
plt.show()

[[ 2.          2.        ]
 [ 2.000002    2.000002  ]
 [ 2.000022    2.000022  ]
 [ 2.00022201  2.00022201]
 [ 2.00222323  2.00222323]
 [ 2.02234591  2.02234591]
 [ 2.23503789  2.23503789]
 [ 2.47009888  2.47009888]
 [ 2.72988144  2.72988144]
 [ 3.01698558  3.01698558]
 [ 3.33428473  3.33428473]
 [ 3.68495451  3.68495451]
 [ 4.07250456  4.07250456]
 [ 4.50081361  4.50081361]
 [ 4.97416831  4.97416831]
 [ 5.49730616  5.49730616]
 [ 6.0754629   6.0754629 ]
 [ 6.71442491  6.71442491]
 [ 7.42058714  7.42058714]
 [ 8.20101711  8.20101711]
 [ 9.06352561  9.06352561]
 [10.01674492 10.01674492]
 [11.07021518 11.07021518]
 [12.23447988 12.23447988]
 [13.52119136 13.52119136]
 [14.94322747 14.94322747]
 [16.51482043 16.51482043]
 [18.25169926 18.25169926]
 [20.17124723 20.17124723]
 [22.29267582 22.29267582]
 [24.63721701 24.63721701]
 [27.22833575 27.22833575]
 [30.09196483 30.09196483]
 [33.2567644  33.2567644 ]
 [36.75440886 36.75440886]
 [40.61990379 40.61990379]
 [44.89193637 44.89193637]
 

/tmp/ipykernel_426399/2237497027.py:68: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [3]:
import pytest
import numpy as np

from regelum.scenario import Scenario
from regelum.simulator import SciPy
from regelum.simulator import CasADi

from src.system import HydraulicSystem
from src.policy import PDController

from regelum import callback

sampling_time = 1e-3

system = HydraulicSystem # HydraulicSystem()

simulator = CasADi( #SciPy(
    system=system,
    state_init = np.array([1e3, 0]),
    time_final = 10e-3,
)

scenario = Scenario(
    policy=PDController(
        system=system,
        sampling_time=sampling_time
    ),
    simulator=simulator,
    sampling_time=0.01,
    N_episodes=1,
    N_iterations=1,
)

scenario.run()

ImportError: cannot import name 'HydraulicSystem' from 'src.system' (/home/kalexu/Projects/AdvContrMethods/Project/src/system.py)

In [1]:
import numpy as np
from src.system import HydraulicSystem
from src.policy import PDController

from regelum.scenario import Scenario
from regelum.simulator import SciPy

# from regelum.system import System
from regelum.utils import rg


system = HydraulicSystem()

simulator = SciPy(
    system=system,
    state_init = np.array([1e3, 0, 0, 0]),
    time_final = 10e-3,
)

AttributeError: 'HydraulicSystem' object has no attribute 'compute_closed_loop_rhs'

In [2]:
system.compute_state_dynamics(0, np.array([2.0, 0, 0]), inputs=np.array([[10]]))

array([[    0.,     0., 10000.]])